<a href="https://colab.research.google.com/github/astromad/MyDeepLearningRepo/blob/master/finalTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ip install --upgrade git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-286yuqzb
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-286yuqzb
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=865978 sha256=402fd365be4a1ab809d86fb5b90210437cda8db46f8bf34868e1367ba200d515
  Stored in directory: /tmp/pip-ephem-wheel-cache-blxjj9sn/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers
  Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [2]:
!pip install transformers

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
    AutoConfig,
    TFBertForTokenClassification,
    BertTokenizer
)

In [4]:
df_data = pd.read_csv("/content/drive/My Drive/ColabData/data_keras.csv",sep=",",encoding="latin1").fillna(method='ffill')
train_df,test_df=train_test_split(df_data,test_size=0.01,shuffle=False)
print (train_df.shape,test_df.shape)
# a list that has all possible labels 
labels =tag_list= train_df['tag'].unique()
label_map =  {i: label for i, label in enumerate(labels)}
num_labels = len(labels)
print(labels) 

(1230145, 3) (12426, 3)
['O' 'I-NAME' 'I-DATE' 'I-SSN' 'I-CCARD' 'I-PHONE']


In [5]:
agg_func = lambda s: [ [w,t] for w,t in zip(s["word"].values.tolist(),s["tag"].values.tolist())]
x_train_grouped = train_df.groupby("sentence_idx").apply(agg_func)
x_test_grouped = test_df.groupby("sentence_idx").apply(agg_func)

x_train_sentences = [[s[0] for s in sent] for sent in x_train_grouped.values]
x_test_sentences = [[s[0] for s in sent] for sent in x_test_grouped.values]

x_train_tags = [[t[1] for t in tag] for tag in x_train_grouped.values]
x_test_tags = [[t[1] for t in tag] for tag in x_test_grouped.values]

In [6]:
model_args = dict()

# Path to pretrained model or model identifier from huggingface.co/models
model_args['model_name'] = 'bert-base-uncased' 

# Where do you want to store the pretrained models downloaded from s3
model_args['cache_dir'] = "CustomNER_cache/"

# we skip basic white-space tokenization by passing do_basic_tokenize = False to the tokenizer
model_args['do_basic_tokenize'] = False


data_args = dict()

data_args['data_dir'] = '/content/drive/My Drive/ColabData/' 

# "The maximum total input sequence length after tokenization. Sequences longer "
# "than this will be truncated, sequences shorter will be padded."
data_args['max_seq_length'] = 512 

# Overwrite the cached training and evaluation sets
# this means the model does not have to tokenize/preprocess and cache the data each time it's called
# this can be made different for each NerDataset (training NerDataset, testing NerDataset)
data_args['overwrite_cache'] = False

In [7]:
config = AutoConfig.from_pretrained(
    model_args['model_name'],
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=model_args['cache_dir']
)

# we skip basic white-space tokenization by passing do_basic_tokenize = False to the tokenizer
tokenizer = BertTokenizer.from_pretrained(
    model_args['model_name'],
    cache_dir=model_args['cache_dir'],
    do_basic_tokenize = model_args['do_basic_tokenize']
)

model = TFBertForTokenClassification.from_pretrained(
    model_args['model_name'],
    config=config,
    cache_dir=model_args['cache_dir']
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForTokenClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
MAX_LENGTH=128
max_seq_length =128
pad_token_label_id = -1
BATCH_SIZE=32
pad_token=0
pad_token_segment_id=0
sequence_a_segment_id=0

In [9]:
def convert_to_input(sentences,tags):
  input_id_list,attention_mask_list,token_type_id_list=[],[],[]
  label_id_list=[]
  
  for x,y in tqdm(zip(sentences,tags),total=len(tags)):
  
    tokens = []
    label_ids = []

    for word, label in zip(x, y):
      word_tokens = tokenizer.tokenize(word)
      #word_tokens = word

      tokens.extend(word_tokens)
      # Use the real label id for the first token of the word, and padding ids for the remaining tokens
      label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))
      #label_ids.extend(
  
    special_tokens_count =  2
    if len(tokens) > max_seq_length - special_tokens_count:
      tokens = tokens[: (max_seq_length - special_tokens_count)]
      label_ids = label_ids[: (max_seq_length - special_tokens_count)]

    label_ids = [pad_token_label_id]+label_ids+[pad_token_label_id]
    inputs = tokenizer.encode_plus(tokens,add_special_tokens=True, max_length=max_seq_length)

    input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
    attention_masks = [1] * len(input_ids)

    attention_mask_list.append(attention_masks)
    input_id_list.append(input_ids)
    token_type_id_list.append(token_type_ids)

    label_id_list.append(label_ids)

  return input_id_list,token_type_id_list,attention_mask_list,label_id_list

In [10]:
import logging
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from tqdm import tqdm,trange
logging.basicConfig(level=logging.ERROR)

label_map = {label: i for i, label in enumerate(tag_list)}
input_ids_train,token_ids_train,attention_masks_train,label_ids_train=convert_to_input(x_train_sentences,x_train_tags)
input_ids_test,token_ids_test,attention_masks_test,label_ids_test=convert_to_input(x_test_sentences,x_test_tags)
input_ids_train = pad_sequences(input_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_train = pad_sequences(token_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_train = pad_sequences(attention_masks_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_train = pad_sequences(label_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

print(np.shape(input_ids_train),np.shape(token_ids_train),np.shape(attention_masks_train),np.shape(label_ids_train))

input_ids_test = pad_sequences(input_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_test = pad_sequences(token_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_test = pad_sequences(attention_masks_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_test = pad_sequences(label_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

print(np.shape(input_ids_test),np.shape(token_ids_test),np.shape(attention_masks_test),np.shape(label_ids_test))

100%|██████████| 399/399 [00:00<00:00, 598.51it/s]


(10000, 128) (10000, 128) (10000, 128) (10000, 128)
(399, 128) (399, 128) (399, 128) (399, 128)


In [11]:
import tensorflow as tf
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y
train_ds = tf.data.Dataset.from_tensor_slices((input_ids_train,attention_masks_train,token_ids_train,label_ids_train)).map(example_to_features)
test_ds=tf.data.Dataset.from_tensor_slices((input_ids_test,attention_masks_test,token_ids_test,label_ids_test)).map(example_to_features)

In [12]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/utils_ner.py

In [13]:
# import utils_ner
# from utils_ner import TFNerDataset, Split, TokenClassificationTask

# train_dataset = TFNerDataset(
#   token_classification_task =TokenClassificationTask,
#   data_dir=data_args['data_dir'],
#   tokenizer=tokenizer,
#   labels=labels,
#   model_type=config.model_type,
#   max_seq_length=data_args['max_seq_length'],
#   overwrite_cache=data_args['overwrite_cache'],
#   mode=Split.train
#   )

In [14]:
from transformers import TFTrainer, TFTrainingArguments,TFDistilBertForSequenceClassification
#tf.compat.v1.disable_v2_behavior()

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_train=True,
    do_eval=False,
    do_predict=False,
    save_total_limit=2
)
with training_args.strategy.scope():
  #model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
  model = TFBertForTokenClassification.from_pretrained(
      model_args['model_name'],
      config=config,
      cache_dir=model_args['cache_dir']
  )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_ds,         # training dataset
    eval_dataset=test_ds             # evaluation dataset

)

In [15]:
#tf.config.experimental_run_functions_eagerly(False)
trainer.train()

AttributeError: ignored

In [ ]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
# model.compile(optimizer=optimizer, loss=model.compute_loss) # can also use any keras loss fn
# #model.train()
# model.fit(train_ds, epochs=3, batch_size=1)

In [ ]:
from transformers import pipeline
nlp_bert_lg = pipeline('ner', model=model, tokenizer=tokenizer,grouped_entities=True)
print(nlp_bert_lg('my name is madhava avvari and my phone number is  408-306-1500 and my ssn is 616-89-2356'))
print(nlp_bert_lg('my name is Murthy Vanka and my phone number is  6502562319 and my ssn is 619231593'))

In [ ]:
model.save_pretrained('./save/')


In [ ]:
sequence = "Hugging Face Inc. 408-306-1500 is a company 616-98-1224 based in New York City."

# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
#tokens = sequence.split()
print(tokens)
inputs = tokenizer.encode_plus(sequence, return_tensors="tf")
print(inputs)
outputs = model(inputs)[0]
predictions = tf.argmax(outputs, axis=2)

print([(token, tag_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())])